In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import torch
import random
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [ ]:
data_root = '../input/plates/plates/'
print(os.listdir(data_root))

In [ ]:
from tqdm import tqdm
train_dir = '../input/plates/plates/train'
class_names = ['cleaned', 'dirty']

In [ ]:
''' Создает итераторы , которые по директориям, которые получают картинки и формируют из эти картинок 
тензоры(батчи с изображениями), которые уже можно передовать в нейронную сеть '''

import torchvision
import matplotlib.pyplot as plt
import time
import copy

from torchvision import transforms, models
train_transforms = transforms.Compose([
    #transforms.CenterCrop(340),
    #transforms.RandomRotation((-45, 45), expand=True, center=(170, 170)),
    #transforms.CenterCrop(224),
    #transforms.CenterCrop(224),
    #torchvision.transforms.RandomAffine((-90, +90), scale=(0.5, 1.5), shear=(-45, +45), resample=False, fillcolor=0),
    #torchvision.transforms.RandomAffine((0, 0), shear=(-45, +45), resample=False, fillcolor=0),
    #transforms.CenterCrop(224),
    transforms.RandomResizedCrop(224),
    transforms.ColorJitter(1.0, 0.3, 0.3, 0.5),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x[np.random.permutation(3), :, :]),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)

batch_size = 8
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)

In [ ]:
len(train_dataloader), len(train_dataset)

In [ ]:
X_batch, y_batch = next(iter(train_dataloader))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
plt.imshow(X_batch[0].permute(1, 2, 0).numpy() * std + mean);

In [ ]:
def show_input(input_tensor, title=''):
    
    np.random.permutation(input_tensor[0])
    print(input_tensor.size())
    image = input_tensor.permute(1, 2, 0).numpy()
    image = std * image + mean
    plt.imshow(image.clip(0, 1))
    plt.title(title)
    plt.show()
    plt.pause(0.001)

X_batch, y_batch = next(iter(train_dataloader))

for x_item, y_item in zip(X_batch, y_batch):
    show_input(x_item, title=class_names[y_item])

In [ ]:
loss_history = []
accuracy_history = []


In [ ]:
def train_model(model, loss, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)
        
        # Each epoch has a training and validation phase
        # for phase in ['train', 'val']:
        # if phase == 'train':
        dataloader = train_dataloader
        scheduler.step()
        model.train()  # Set model to training mode
            #else:
                #dataloader = val_dataloader
                #model.eval()   # Set model to evaluate mode

        running_loss = 0.
        running_acc = 0.
        #running_loss = []
        #running_acc = []

        # Iterate over data.
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

                # forward and backward
            #with torch.set_grad_enabled(True):  #  указывает на то что на валидации нам не нужны подсчеты градиента
            preds = model(inputs)
            loss_value = loss(preds, labels)
            preds_class = preds.argmax(dim=1)

                    # backward + optimize only if in training phase
                #if phase == 'train':
            loss_value.backward()
            optimizer.step()

                # statistics
            running_loss += loss_value.item()
            running_acc += (preds_class == labels.data).float().mean()
            #print('running_loss', running_loss)
            #print('running_acc', running_acc )

        epoch_loss = running_loss / len(dataloader)
        loss_history.append(epoch_loss)
        epoch_acc = running_acc / len(dataloader)
        accuracy_history.append(epoch_acc)

        print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc), flush=True)

    return model

In [ ]:
model = models.resnet152(pretrained=True)

# Disable grad for all conv layers
for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Linear(model.fc.in_features, 2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), amsgrad=True, lr=1.0e-3, weight_decay=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
train_model(model, loss, optimizer, scheduler, num_epochs=120);

In [ ]:
import shutil 
test_dir = 'test'
shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))

In [ ]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    
test_dataset = ImageFolderWithPaths(test_dir, val_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

In [ ]:
model.eval()

test_predictions = []
test_img_paths = []
for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs)
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    test_img_paths.extend(paths)
    
test_predictions = np.concatenate(test_predictions)

In [ ]:
inputs, labels, paths = next(iter(test_dataloader))

for img, pred in zip(inputs, test_predictions):
    show_input(img, title=pred)

In [ ]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})

In [ ]:
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.5 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=6)

In [ ]:
submission_df.to_csv('submission.csv')

In [ ]:
!rm -rf train val test

In [ ]:
plt.plot(loss_history)

In [ ]:
plt.plot(accuracy_history)